In [ ]:
import csv
import re
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:


# Replace with your Twitter login credentials
username = "username"
password = "Password"
organisationname = "Essex_CC"

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# Open Twitter login page
driver.get("https://twitter.com/"+organisationname)

# Wait for the page to load
time.sleep(2)

# Find the username and password input fields and fill them
username_input = driver.find_element(By.NAME, "text")
username_input.send_keys(username)

# Wait for the "Next" button to be clickable
next_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Next")]'))
)
next_button.click()

# Wait for the page to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "password")))

# Find the password input field by its name attribute and fill it with the password
password_input = driver.find_element(By.NAME,"password")
password_input.send_keys(password)

login_button = driver.find_element(By.CSS_SELECTOR, '[data-testid="LoginForm_Login_Button"]')
login_button.click()

# Wait for the login process to complete
time.sleep(5)



In [ ]:
def get_tweet_data(tweet):
   user = tweet.find_element(By.XPATH,'.//div[@data-testid="User-Name"]//span').text
   tweet_handle = tweet.find_element(By.XPATH,'.//div[@dir="ltr"]//span[contains(text(),"@")]').text
   try:
      tweet_time = tweet.find_element(By.XPATH,'.//time').get_attribute('datetime')
   except NoSuchElementException:
      return
   comment = tweet.find_element(By.XPATH, './/div[1]/div[1]/div[2]/div[2]/div[2]').text
   response = tweet.find_element(By.XPATH, './/div[1]/div[1]/div[2]/div[2]/div[3]').text
   text = comment + response
   reply_cnt = tweet.find_element(By.XPATH, './/div[1]/div[1]/div[2]/div[2]//div[@data-testid="reply"]').text
   retweet_cnt = tweet.find_element(By.XPATH, './/div[1]/div[1]/div[2]/div[2]//div[@data-testid="retweet"]').text
   like_cnt = tweet.find_element(By.XPATH, './/div[1]/div[1]/div[2]/div[2]//div[@data-testid="like"]').text

   tweet_data = (user,tweet_handle,tweet_time,text,reply_cnt,retweet_cnt,like_cnt)
   return tweet_data



In [ ]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH,'//article[@data-testid="tweet"]')
    for card in page_cards[3:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                print("printing tweet id: ",tweet_id)
                tweet_ids.add(tweet_id)
                data.append(tweet)
                print(data)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(3)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(3) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
# driver.close()

In [ ]:
with open('EssexCountyCouncilData.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header row
    header = ['UserName', 'Handle', 'Date_Time', 'Tweet', 'Reply', 'Retweet', 'Likes']
    writer.writerow(header)
    
    # Iterate over the data
    for item in data:
        # Replace newline characters with spaces in the tweet text
        cleaned_tweet = item[3].replace('\n', ' ')
        
        # Write the cleaned tweet data to the CSV file
        writer.writerow([
            item[0],  # UserName
            item[1],  # Handle
            item[2],  # Date_Time
            cleaned_tweet,  # Tweet
            item[4],  # Reply
            item[5],  # Retweet
            item[6]   # Likes
        ])